In [37]:
from pandas import read_excel, pivot_table, melt, merge, DataFrame
import numpy as np
import re

In [4]:
origin = read_excel("MusinsaSampleData.xlsx")
df = origin.copy()
df

,상품명,종류,분류,성별,조회수,누적판매,선호연령,선호연령비율,선호성별,남자선호비율,여자선호비율,상품가격,제품소재,색상
0,와이드 데님 팬츠 [그레이],바지,데님 팬츠,"남성, 여성",144000,8500,"23세 이하,","['~18세42,19~23세21,24~28세12,29~33세7,34~39세4,40세...",남성,0.75,0.25,21200.0,COTTON 100%,"BLACK, GRAY, RAW DENIM, MEDIUM BLUE"
1,1992 오버핏 스웨트셔츠 화이트멜란지,상의,맨투맨/스웨트셔츠,NaN,1400,163,0,"['~18세49,19~23세3,24~28세6,29~33세10,34~39세9,40세~...",NaN,0.49,0.51,36900.0,상세페이지 참조,상세페이지 참조
2,핸드볼 스페지알 - 핑크 / IF6561,스니커즈,패션스니커즈화,"남성, 여성",79000,1600,"24 33세,","['~18세18,19~23세16,24~28세24,29~33세18,34~39세11,4...",여성,0.34,0.66,139000.0,상품 Tag 참고,핑크
3,베이직 푸퍼_SPJPE11C11,아우터,숏패딩/숏헤비 아우터,남성,608000,75000,"19 28세,","['~18세57,19~23세14,24~28세10,29~33세6,34~39세4,40세...",남성,0.73,0.27,62910.0,겉감 폴리에스터100%안감 폴리에스터100%,"(10)WHITE,(15)GRAY,(16)LIGHT GRAY,(19)BLACK,(3..."
4,워셔블 아란 케이블 후드 집업 니트 (IVORY),상의,니트/스웨터,"남성, 여성",186000,4600,"19 28세,","['~18세24,19~23세27,24~28세23,29~33세10,34~39세5,40...",여성,0.36,0.64,39900.0,Acryl 80%Nylon 20%,아이보리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,남성드레스셔츠 네이비 솔리드 슬림핏 1THTHA4MSU105,상의,셔츠/블라우스,NaN,40,0,0,['~18세19~23세24~28세29~33세34~39세40세~'],NaN,0.92,0.08,35000.0,"폴리에스터 45%, 면 55%",파란색
7658,컬러 블록 후드 블랙,상의,후드 티셔츠,NaN,314,0,0,"['~18세21,19~23세20,24~28세6,29~33세18,34~39세9,40세...",NaN,0.40,0.60,31000.0,상품 상세설명 참조,블랙
7659,X 버클캡 [그린],모자,캡/야구 모자,공용,700,150,"29 39세,","['~18세26,19~23세11,24~28세11,29~33세17,34~39세13,4...",남성,0.62,0.38,35100.0,상세페이지 참조,Cotton 100%
7660,X 버클캡 [베이지],모자,캡/야구 모자,공용,900,200,"24 33세,","['~18세38,19~23세13,24~28세14,29~33세16,34~39세10,4...",남성,0.50,0.50,35100.0,상세페이지 참조,Cotton 100%


Step 1 - 무신사 연령별 데이터 정리

In [39]:
# 연령 별 컬럼 생성
df1 = df.filter(['상품명','선호연령비율'])
df1['~18세'] =""
df1['19~23세']=""
df1['24~28세']=""
df1['29~33세']=""
df1['34~39세']=""
df1['40세~']=""

# 선호연령비율 컬럼 내용을 리스트에서 문자열로 변환
raw=df1['선호연령비율']
for i in range(len(df1.index)):
    raw[i]=raw[i].strip("[]")

for j in range(len(df1.index)):
    try:
        input_str = raw[j]

# 정규표현식을 사용하여 숫자와 연령대를 추출
        matches = re.findall(r'(\d+)?~?(\d+)세~?(\d+)?,', input_str)

# 추출된 정보를 리스트에 저장
        result = [int(match[2]) if match[2] else (int(match[0]) if match[0] else int(match[1])) for match in matches]
        raw[j] = result
    except Exception as e:
        print("예외발생", e)
error = []
for i in range(len(df1.index)):
    if len(raw[i]) <6:
        error.append(i)
print("error index:", error, "총 error:", len(error))

for j in range(len(df1.index)):
    if j in error:
        pass
    else:
        df1['~18세'][j] =raw[j][0]
        df1['19~23세'][j]=raw[j][1]
        df1['24~28세'][j]=raw[j][2]
        df1['29~33세'][j]=raw[j][3]
        df1['34~39세'][j]=raw[j][4]
        df1['40세~'][j]=raw[j][5]

error index: [56, 133, 306, 318, 417, 585, 641, 642, 756, 759, 763, 798, 800, 801, 816, 817, 826, 827, 831, 845, 861, 898, 907, 933, 966, 1006, 1028, 1031, 1038, 1051, 1104, 1114, 1127, 1137, 1154, 1165, 1177, 1205, 1206, 1207, 1208, 1209, 1213, 1227, 1231, 1242, 1247, 1251, 1275, 1276, 1290, 1292, 1294, 1295, 1299, 1313, 1326, 1336, 1340, 1370, 1371, 1372, 1382, 1415, 1418, 1426, 1433, 1439, 1445, 1454, 1478, 1502, 1504, 1505, 1518, 1531, 1534, 1536, 1537, 1538, 1539, 1549, 1551, 1560, 1567, 1569, 1573, 1581, 1585, 1605, 1606, 1608, 1609, 1611, 1612, 1634, 1639, 1665, 1671, 1685, 1694, 1698, 1709, 1710, 1717, 1735, 1736, 1737, 1768, 1780, 1787, 1790, 1792, 1801, 1805, 1806, 1812, 1841, 1852, 1868, 1871, 1876, 1905, 1915, 1933, 1964, 1967, 1968, 2008, 2009, 2037, 2060, 2067, 2068, 2093, 2102, 2113, 2122, 2124, 2125, 2133, 2149, 2156, 2169, 2170, 2175, 2176, 2185, 2208, 2216, 2226, 2232, 2265, 2273, 2276, 2277, 2283, 2328, 2330, 2346, 2352, 2362, 2366, 2374, 2391, 2395, 2397, 2399, 2400

Step 2 - 연령별 데이터 기존 데이터와 병합

In [38]:
merged = merge(df, df1, on = '상품명')

Step 3 - 정리된 데이터 엑셀로 내보내기

In [ ]:
merged.to_excel("MusinsaSampleData_age.xlsx", index=False)

기타 시도들 및 테스트 결과값들

In [22]:
df1 = origin.filter(['상품명','선호연령비율'])
df1['~18세'] =""
df1['19~23세']=""
df1['24~28세']=""
df1['29~33세']=""
df1['34~39세']=""
df1['40세~']=""
df1
raw=df1['선호연령비율']
for i in range(len(df1.index)):
    raw[i]=raw[i].strip("[]")

for j in range(len(df1.index)):
    input_str = raw[j]

# 정규표현식을 사용하여 숫자와 연령대를 추출
    matches = re.findall(r'(\d+)?~?(\d+)세~?(\d+)?,', input_str)

# 추출된 정보를 리스트에 저장
    result = [int(match[2]) if match[2] else (int(match[0]) if match[0] else int(match[1])) for match in matches]
    raw[j] = result
error = []
for i in range(len(df1.index)):
    if len(raw[i]) <6:
        error.append(i)

for j in range(len(df1.index)):
    if j in error:
        pass
    else:
        df1['~18세'][j] =raw[j][0]
        df1['19~23세'][j]=raw[j][1]
        df1['24~28세'][j]=raw[j][2]
        df1['29~33세'][j]=raw[j][3]
        df1['34~39세'][j]=raw[j][4]
        df1['40세~'][j]=raw[j][5]


print("error index:", error, "총 error:", len(error))


error index: [56, 133, 306, 318, 417, 585, 641, 642, 756, 759, 763, 798, 800, 801, 816, 817, 826, 827, 831, 845, 861, 898, 907, 933, 966, 1006, 1028, 1031, 1038, 1051, 1104, 1114, 1127, 1137, 1154, 1165, 1177, 1205, 1206, 1207, 1208, 1209, 1213, 1227, 1231, 1242, 1247, 1251, 1275, 1276, 1290, 1292, 1294, 1295, 1299, 1313, 1326, 1336, 1340, 1370, 1371, 1372, 1382, 1415, 1418, 1426, 1433, 1439, 1445, 1454, 1478, 1502, 1504, 1505, 1518, 1531, 1534, 1536, 1537, 1538, 1539, 1549, 1551, 1560, 1567, 1569, 1573, 1581, 1585, 1605, 1606, 1608, 1609, 1611, 1612, 1634, 1639, 1665, 1671, 1685, 1694, 1698, 1709, 1710, 1717, 1735, 1736, 1737, 1768, 1780, 1787, 1790, 1792, 1801, 1805, 1806, 1812, 1841, 1852, 1868, 1871, 1876, 1905, 1915, 1933, 1964, 1967, 1968, 2008, 2009, 2037, 2060, 2067, 2068, 2093, 2102, 2113, 2122, 2124, 2125, 2133, 2149, 2156, 2169, 2170, 2175, 2176, 2185, 2208, 2216, 2226, 2232, 2265, 2273, 2276, 2277, 2283, 2328, 2330, 2346, 2352, 2362, 2366, 2374, 2391, 2395, 2397, 2399, 2400

In [5]:
raw=df1['선호연령비율']
for i in range(len(df1.index)):
    raw[i]=raw[i].strip("[]")
raw

0       '~18세58,19~23세8,24~28세7,29~33세12,34~39세8,40세~7,'
1      '~18세46,19~23세7,24~28세16,29~33세17,34~39세7,40세~7,'
2      '~18세17,19~23세23,24~28세26,29~33세18,34~39세8,40세...
3      '~18세16,19~23세29,24~28세29,29~33세13,34~39세6,40세...
4      '~18세26,19~23세15,24~28세21,29~33세20,34~39세10,40...
                             ...                        
818    '~18세22,19~23세33,24~28세33,29~33세8,34~39세2,40세~2,'
819    '~18세27,19~23세25,24~28세21,29~33세11,34~39세5,40세...
820                                                     
821    '~18세48,19~23세24,24~28세14,29~33세5,34~39세2,40세~7,'
822    '~18세20,19~23세21,24~28세23,29~33세17,34~39세10,40...
Name: 선호연령비율, Length: 823, dtype: object

In [6]:

for j in range(len(df1.index)):
    input_str = raw[j]

# 정규표현식을 사용하여 숫자와 연령대를 추출
    matches = re.findall(r'(\d+)?~?(\d+)세~?(\d+)?,', input_str)

# 추출된 정보를 리스트에 저장
    result = [int(match[2]) if match[2] else (int(match[0]) if match[0] else int(match[1])) for match in matches]
    raw[j] = result


In [7]:
error = []
for i in range(len(df1.index)):
    if len(raw[i]) <6:
        error.append(i)
len(error)

61

### [4]

In [8]:
for j in range(len(df1.index)):
    if j in error:
        pass
    else:
        df1['~18세'][j] =raw[j][0]
        df1['19~23세'][j]=raw[j][1]
        df1['24~28세'][j]=raw[j][2]
        df1['29~33세'][j]=raw[j][3]
        df1['34~39세'][j]=raw[j][4]
        df1['40세~'][j]=raw[j][5]


In [9]:
df1.head(22)

,상품명,선호연령비율,~18세,19~23세,24~28세,29~33세,34~39세,40세~
0,아디컬러 Neuclassics 재킷 HR8696,"[58, 8, 7, 12, 8, 7]",58,8,7,12,8,7
1,NJ3BP05A 파이어 쉴드 자켓,"[46, 7, 16, 17, 7, 7]",46,7,16,17,7,7
2,4.5cm 소가죽 코만도 첼시부츠 루틴(CR0011),"[17, 23, 26, 18, 8, 8]",17,23,26,18,8,8
3,Butter bag Classic · 버터백 클래식,"[16, 29, 29, 13, 6, 7]",16,29,29,13,6,7
4,LEATHER PUFFER [BLACK],"[26, 15, 21, 20, 10, 8]",26,15,21,20,10,8
5,왓이브 후드 네이비 HHHD3472,"[52, 16, 9, 6, 3, 14]",52,16,9,6,3,14
6,오버핏 블룸 후디-블랙,"[18, 13, 21, 19, 12, 17]",18,13,21,19,12,17
7,에코 퍼 플리스 숏 무스탕 자켓 점퍼 [베이지],"[15, 26, 28, 14, 7, 10]",15,26,28,14,7,10
8,오버사이즈 멜톤 콘보이 더플 코트 네이비,"[33, 30, 17, 7, 3, 10]",33,30,17,7,3,10
9,스웨이드 오버핏 양털 무톤 무스탕 자켓,"[26, 38, 20, 8, 3, 5]",26,38,20,8,3,5


re.findall() 함수는 정규 표현식 패턴과 일치하는 모든 부분 문자열을 찾아 리스트로 반환하는 Python의 re 모듈 함수입니다. 여기서는 주어진 문자열에서 숫자와 연령대를 추출하는 데 사용됩니다.

이 정규 표현식은 세 개의 그룹으로 나뉘어 있습니다:

(\d+)?: 0 또는 그 이상의 숫자(\d+)를 나타내는 그룹입니다. ?는 이 그룹이 0 또는 1회 나타날 수 있음을 나타냅니다.
~?: ~ 문자가 0 또는 1회 나타날 수 있는 그룹입니다. ~ 문자가 있거나 없을 수 있습니다.
(\d+)?: 또 다른 숫자를 나타내는 그룹입니다.
정규 표현식 전체는 숫자, '~' 문자, 숫자의 조합을 찾아내는 패턴입니다. 여러 그룹이 있기 때문에 findall()은 각 매치에 대해 튜플의 리스트를 반환합니다. 예를 들어 '~18세58'의 경우 findall()은 [(None, '18', '58')]와 같은 리스트를 반환합니다.

그런 다음 코드는 이러한 튜플을 반복하면서 각 경우에 따라 숫자를 추출하고 결과 리스트에 저장합니다.